<a href="https://colab.research.google.com/github/rajesh-bhat/data-aisummit-2021-databricks-conversational-ai/blob/main/TF2_Intent_Classification_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu May 27 17:21:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install tqdm
!pip3 install transformers

     |████████████████████████████████| 2.3MB 7.7MB/s 
     |████████████████████████████████| 901kB 43.4MB/s 
     |████████████████████████████████| 3.3MB 48.1MB/s 


## Downloading the data

In [3]:
!gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output train.csv
!gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output valid.csv
!gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output test.csv

Downloading...
From: https://drive.google.com/uc?id=1OlcvGWReJMuyYQuOZm149vHWwPtlboR6
To: /content/train.csv
100% 799k/799k [00:00<00:00, 52.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w
To: /content/valid.csv
100% 43.3k/43.3k [00:00<00:00, 67.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF
To: /content/test.csv
100% 43.1k/43.1k [00:00<00:00, 41.6MB/s]


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer
%matplotlib inline

# fix random seed for reproducibility
seed = 222
np.random.seed(seed)
tf.random.set_seed(seed)

print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF Version:  2.5.0
Eager mode:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


##Overview of data

In [5]:
train = pd.concat([pd.read_csv(file) for file in ["train.csv"]])
train = train.groupby('intent').sample(frac=0.25)

val = pd.read_csv("valid.csv")

print(train.shape)
print(val.shape)

(3270, 2)
(700, 2)


In [6]:
train.head()

,text,intent
4024,can you put this tune onto latin dance cardio,AddToPlaylist
4592,marcia carolus rex should get added to my wome...,AddToPlaylist
9153,add r u still in 2 it to the under the surface...,AddToPlaylist
11241,add a billy strayhorn song to my the selektor ...,AddToPlaylist
8989,add an album to women of hip hop,AddToPlaylist


In [7]:
val.head()

,text,intent
0,i d like to have this track onto my classical ...,AddToPlaylist
1,add the album to my flow español playlist,AddToPlaylist
2,add digging now to my young at heart playlist,AddToPlaylist
3,add this song by too poetic to my piano ballad...,AddToPlaylist
4,add this album to old school death metal,AddToPlaylist


In [8]:
train.intent.value_counts()

PlayMusic               478
GetWeather              474
BookRestaurant          470
RateBook                469
SearchScreeningEvent    463
SearchCreativeWork      462
AddToPlaylist           454
Name: intent, dtype: int64

## Label Encoded target variables

In [35]:
intent_mapping = {x:idx for idx,x in enumerate(train.intent.unique().tolist())}
intent_mapping_rev = {v: k for k, v in intent_mapping.items()}

train['target'] = train['intent'].map(intent_mapping)
val['target'] = val['intent'].map(intent_mapping)

train.head()

,text,intent,target
4024,can you put this tune onto latin dance cardio,AddToPlaylist,0
4592,marcia carolus rex should get added to my wome...,AddToPlaylist,0
9153,add r u still in 2 it to the under the surface...,AddToPlaylist,0
11241,add a billy strayhorn song to my the selektor ...,AddToPlaylist,0
8989,add an album to women of hip hop,AddToPlaylist,0


In [36]:
intent_mapping, intent_mapping_rev

({'AddToPlaylist': 0,
  'BookRestaurant': 1,
  'GetWeather': 2,
  'PlayMusic': 3,
  'RateBook': 4,
  'SearchCreativeWork': 5,
  'SearchScreeningEvent': 6},
 {0: 'AddToPlaylist',
  1: 'BookRestaurant',
  2: 'GetWeather',
  3: 'PlayMusic',
  4: 'RateBook',
  5: 'SearchCreativeWork',
  6: 'SearchScreeningEvent'})

In [12]:
val.head()    

,text,intent,target
0,i d like to have this track onto my classical ...,AddToPlaylist,0
1,add the album to my flow español playlist,AddToPlaylist,0
2,add digging now to my young at heart playlist,AddToPlaylist,0
3,add this song by too poetic to my piano ballad...,AddToPlaylist,0
4,add this album to old school death metal,AddToPlaylist,0


## One hot encoding the labels

In [16]:
lb = LabelBinarizer( )
lb.fit(train["target"])
print(lb.classes_)

y_train = lb.transform(train["target"])
y_val = lb.transform(val["target"])

[0 1 2 3 4 5 6]


In [32]:
def create_bert_input_features(tokenizer, docs, max_seq_length):
    '''
    tokenizer: based on the model chosen
    docs: sentences/utterances
    max_seq_length: used for padding
    '''

    all_ids, all_masks = [], []

    for doc in tqdm.tqdm(docs, desc="Converting sentences to features"):
        
        tokens = tokenizer.tokenize(doc)

        if len(tokens) > max_seq_length-2:
            tokens = tokens[0 : (max_seq_length-2)]
        
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        ids = tokenizer.convert_tokens_to_ids(tokens)
        masks = [1] * len(ids)

        # zero-padding up to the sequence length.
        while len(ids) < max_seq_length:
            ids.append(0)
            masks.append(0)
        
        all_ids.append(ids)
        all_masks.append(masks)
    
    ## encoded sentence along with masks.
    encoded = np.array([all_ids, all_masks])

    return encoded

## Load DistilBertTokenizer from transformers module

In [14]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

## Model parameters and summary

In [23]:
MAX_SEQ_LENGTH = 256 ## can be chosen based on the sentence length distribution
num_labels = 7 ## with the current data we have 7 classes

inp_id = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_ids")
inp_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name="bert_input_masks")
inputs = [inp_id, inp_mask]

hidden_state = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')(inputs)[0]
pooled_output = hidden_state[:, 0]

## dense layers
dense1 = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
drop1 = tf.keras.layers.Dropout(0.25)(dense1)
dense2 = tf.keras.layers.Dense(256, activation='relu')(drop1)
drop2 = tf.keras.layers.Dropout(0.25)(dense2)
output = tf.keras.layers.Dense(num_labels, activation='softmax')(drop2)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=2e-5, 
                                           epsilon=1e-08), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
bert_input_masks (InputLayer)   [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti TFBaseModelOutput(la 66362880    bert_input_ids[0][0]             
                                                                 bert_input_masks[0][0]           
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           tf_distil_bert_model_1[0][0

In [24]:
train_features_ids, train_features_masks = create_bert_input_features(tokenizer, train["text"], max_seq_length=MAX_SEQ_LENGTH)
val_features_ids, val_features_masks = create_bert_input_features(tokenizer, val["text"], max_seq_length=MAX_SEQ_LENGTH)

print('Train Features:', train_features_ids.shape, train_features_masks.shape)
print('Val Features:', val_features_ids.shape, val_features_masks.shape)

Converting docs to features: 100%|██████████| 700/700 [00:00<00:00, 3393.64it/s]

Train Features: (3270, 256) (3270, 256)
Val Features: (700, 256) (700, 256)


In [25]:
print("Target shape", y_train.shape, y_val.shape)

Target shape (3270, 7) (700, 7)


In [27]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=3,
                                      restore_best_weights=True,
                                      verbose=1)

checkpoint = tf.keras.callbacks.ModelCheckpoint('tf_intent_classification.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')

model.fit([train_features_ids, 
           train_features_masks], y_train, 
          validation_data=([val_features_ids, 
                            val_features_masks], y_val),
          epochs=3, 
          batch_size=32, 
          shuffle=True,
          callbacks=[es],
          verbose=1)

Epoch 1/3
103/103 [==============================] - 86s 835ms/step - loss: 0.1064 - accuracy: 0.9719 - val_loss: 0.0738 - val_accuracy: 0.9771
Epoch 2/3
103/103 [==============================] - 86s 834ms/step - loss: 0.0538 - accuracy: 0.9902 - val_loss: 0.1089 - val_accuracy: 0.9700
Epoch 3/3
103/103 [==============================] - 86s 831ms/step - loss: 0.0354 - accuracy: 0.9930 - val_loss: 0.0791 - val_accuracy: 0.9800


In [37]:
def score(sentence):
    '''
    input:sentence
    return: intent
    '''

    features_ids, features_masks = create_bert_input_features(tokenizer, [sentence], max_seq_length=MAX_SEQ_LENGTH)
    predictions = model.predict([features_ids, features_masks], batch_size=1, verbose=0)
    print(predictions)
    print(np.argmax(predictions))

    return {"query":sentence, "intent":intent_mapping_rev[np.argmax(predictions)]}

In [39]:
score("book a restaurant near my office")

Converting sentences to features: 100%|██████████| 1/1 [00:00<00:00, 943.60it/s]

[[1.4433196e-04 9.9869776e-01 4.9405068e-04 3.5746428e-04 1.8700135e-04
  5.1930572e-05 6.7458204e-05]]
1


{'intent': 'BookRestaurant', 'query': 'book a restaurant near my office'}

References:
<br>
[1] Sarkar, Dipanjan (DJ). DipanjanS/Deep_transfer_learning_nlp_dhs2019. 2019. 2021. GitHub, https://github.com/dipanjanS/deep_transfer_learning_nlp_dhs2019